In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/Flight delays/flight_delays_train.csv")
df_2 = pd.read_csv("../data/Flight delays/flight_delays_test.csv")

In [3]:
# df = pd.concat([df_1, df_2])

In [4]:
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
len(df["Distance"].unique())

1310

In [6]:
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [7]:
df["UniqueCarrier"].unique()

array(['AA', 'US', 'XE', 'OO', 'WN', 'NW', 'DL', 'OH', 'AS', 'UA', 'MQ',
       'CO', 'EV', 'DH', 'YV', 'F9', 'AQ', 'TZ', 'HP', 'B6', 'FL', 'HA'],
      dtype=object)

In [8]:
df["Origin"].unique()

array(['ATL', 'PIT', 'RDU', 'DEN', 'MDW', 'MEM', 'PBI', 'MSP', 'ONT',
       'BDL', 'PHX', 'LAS', 'DFW', 'DSM', 'CMH', 'ORF', 'SLC', 'CLT',
       'GSO', 'IAD', 'SMF', 'FLL', 'DAL', 'ORD', 'ITO', 'SAN', 'ROA',
       'LGA', 'SFO', 'GSP', 'SEA', 'DAB', 'SJC', 'LIT', 'LAX', 'OAK',
       'COS', 'OKC', 'GRR', 'JFK', 'BOI', 'MCI', 'BWI', 'BHM', 'CRP',
       'BOS', 'SAT', 'PHL', 'STL', 'CIC', 'AUS', 'IAH', 'COD', 'HNL',
       'RNO', 'BNA', 'TPA', 'MIA', 'EVV', 'PNS', 'EWR', 'RSW', 'ANC',
       'SNA', 'AMA', 'CID', 'DTW', 'DCA', 'LGB', 'MAF', 'MFE', 'BMI',
       'PDX', 'IPL', 'GRB', 'FAR', 'HOU', 'MTJ', 'DRO', 'MLU', 'VPS',
       'TUL', 'CVG', 'SBA', 'PWM', 'IDA', 'MCO', 'ACV', 'CHS', 'BGM',
       'MSY', 'OGG', 'CLE', 'MOB', 'CAK', 'FAY', 'SHV', 'TUS', 'IND',
       'CAE', 'PVD', 'ROC', 'MFR', 'VLD', 'ELP', 'RIC', 'MKE', 'SGF',
       'TYS', 'CHO', 'EGE', 'BIS', 'JAN', 'JAX', 'BUF', 'MSO', 'BGR',
       'CEC', 'ICT', 'MYR', 'ALB', 'LIH', 'SBP', 'AEX', 'GNV', 'SAV',
       'BTM', 'BRO',

In [9]:
df.isna().mean()

Month                0.0
DayofMonth           0.0
DayOfWeek            0.0
DepTime              0.0
UniqueCarrier        0.0
Origin               0.0
Dest                 0.0
Distance             0.0
dep_delayed_15min    0.0
dtype: float64

In [10]:
def engineering(df):   
    
    df["Month"] = df["Month"].str.replace("c-", "")
    df["DayofMonth"] = df["DayofMonth"].str.replace("c-", "")
    df["DayOfWeek"] = df["DayOfWeek"].str.replace("c-", "")
    
    df["Month"] = df["Month"].astype(int)
    df["DayOfWeek"] = df["DayOfWeek"].astype(int)
    df["DayofMonth"] = df["DayofMonth"].astype(int)
    
    df["DepTime"] = df["DepTime"].astype(str).str.zfill(4)
    df["Time"] = pd.to_datetime(df["DepTime"], format="%H%M", errors="coerce")
    df["Time"] = df["Time"].fillna("00:00")
    
    df["Summer"] = df["Month"].isin([6, 7, 8])
    df["Autumn"] = df["Month"].isin([9, 10, 11])
    df["Winter"] = df["Month"].isin([12, 1, 2])
    df["Spring"] = df["Month"].isin([3, 4, 5])
    
    df.replace({"True": 1, "False": 0}, inplace=True)
    df["Hour"] = df["Time"].dt.hour
    df["Minute"] = df["Time"].dt.minute
    
    df.drop(["DepTime", "Time"], axis=1, inplace=True)

    return df

In [11]:
df = engineering(df)

In [12]:
df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Summer,Autumn,Winter,Spring,Hour,Minute
0,8,21,7,AA,ATL,DFW,732,N,True,False,False,False,19,34
1,4,20,3,US,PIT,MCO,834,N,False,False,False,True,15,48
2,9,2,5,XE,RDU,CLE,416,N,False,True,False,False,14,22
3,11,25,6,OO,DEN,MEM,872,N,False,True,False,False,10,15
4,10,7,6,WN,MDW,OMA,423,Y,False,True,False,False,18,28


In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
df.shape

(100000, 14)

In [15]:
df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Summer,Autumn,Winter,Spring,Hour,Minute
0,8,21,7,AA,ATL,DFW,732,N,True,False,False,False,19,34
1,4,20,3,US,PIT,MCO,834,N,False,False,False,True,15,48
2,9,2,5,XE,RDU,CLE,416,N,False,True,False,False,14,22
3,11,25,6,OO,DEN,MEM,872,N,False,True,False,False,10,15
4,10,7,6,WN,MDW,OMA,423,Y,False,True,False,False,18,28


In [16]:
cat_labels = [c for c in df if not pd.api.types.is_numeric_dtype(df[c]) and c != "dep_delayed_15min"]

In [17]:
cat_labels

['UniqueCarrier', 'Origin', 'Dest']

In [18]:
cat_imputer = Pipeline([
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

In [19]:
preprocessor = ColumnTransformer([
    ("imputer", cat_imputer, cat_labels)
], remainder="passthrough")

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X = df.drop("dep_delayed_15min", axis=1)
y = df["dep_delayed_15min"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [23]:
from catboost import CatBoostClassifier

In [24]:
model = CatBoostClassifier(verbose=False)

In [25]:
model.fit(X_train_preprocessed, y_train, 
          early_stopping_rounds=25, verbose=False, 
          plot=True, eval_set=(X_test_preprocessed, y_test));

ValueError: invalid literal for int() with base 10: 'Y'

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_preds_proba = model.predict_proba(X_test_preprocessed)[:, 1]

In [ ]:
roc_auc_score(y_test, y_preds_proba)